# Scraping and Analysis Tasks

* Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/)
* Collect the latest News Title and Paragraph Text
* Assign the text to variables that you can reference later.

### Data Sources: 

   #### All the data were scraped from the following websites:

* [NASA Mars News Site](https://mars.nasa.gov/news/) -Scraped the latest News Title and Paragraph Text
* [JPL Featured Space Image](https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars) - Scraped the image url for the current Featured Mars Image
* [Mars Weather twitter account](https://twitter.com/marswxreport?lang=en) - Scraped the latest Mars weather tweet
* [USGS Astrogeology site](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) - Scraped high resolution images for each of Mar's hemispheres
* [Mars Facts webpage](https://space-facts.com/mars/) -Scraped the table containing facts about the planet including Diameter, Mass, etc

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from pprint import pprint

import pandas as pd
from splinter import Browser
import time
